## Load the model

In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import xml.etree.ElementTree as ET

from skimage.io import imread
from skimage.transform import resize
from PIL import Image
from imgaug import augmenters as iaa

import os
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.utils import *
from keras.callbacks import *

from keras.applications.densenet import DenseNet121, preprocess_input
# load and evaluate a saved model
from keras.models import load_model
# load model
model = load_model('weights.h5')
# summarize model.
model.summary()

In [ ]:
# Loading the data and preprocessing
breeds = os.listdir("dogs/images/Images/")

num_breeds = len(breeds)

# Mapping
label_maps = {}
label_maps_rev = {}
for i, v in enumerate(breeds):
    label_maps.update({v: i})
    label_maps_rev.update({i : v})
    
def label_target_path():
    paths = list()
    labels = list()
    targets = list()
    for breed in breeds:
        base_name = "./data/{}/".format(breed)
        for img_name in os.listdir(base_name):
            paths.append(base_name + img_name)
            labels.append(breed)
            targets.append(label_maps[breed])
    return paths, labels, targets

paths, labels, targets = label_target_path()

assert len(paths) == len(labels)
assert len(paths) == len(targets)

targets = np_utils.to_categorical(targets, num_classes=num_breeds)

batch_size = 64

class ImageGenerator(Sequence):
    
    def __init__(self, paths, targets, batch_size, shape, augment=False):
        self.paths = paths
        self.targets = targets
        self.batch_size = batch_size
        self.shape = shape
        self.augment = augment
        
    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_paths = self.paths[idx * self.batch_size : (idx + 1) * self.batch_size]
        x = np.zeros((len(batch_paths), self.shape[0], self.shape[1], self.shape[2]), dtype=np.float32)
        y = np.zeros((self.batch_size, num_breeds, 1))
        for i, path in enumerate(batch_paths):
            x[i] = self.__load_image(path)
        y = self.targets[idx * self.batch_size : (idx + 1) * self.batch_size]
        return x, y
    
    def __iter__(self):
        for item in (self[i] for i in range(len(self))):
            yield item
            
    def __load_image(self, path):
        image = imread(path)
        image = preprocess_input(image)
        if self.augment:
            seq = iaa.Sequential([
                iaa.OneOf([
                    iaa.Fliplr(0.5),
                    iaa.Flipud(0.5),
                    iaa.CropAndPad(percent=(-0.25, 0.25)),
                    iaa.Crop(percent=(0, 0.1)),
                    iaa.Sometimes(0.5,
                        iaa.GaussianBlur(sigma=(0, 0.5))
                    ),
                    iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-180, 180),
                        shear=(-8, 8)
                    )
                ])
            ], random_order=True)
            image = seq.augment_image(image)
        return image
    
X_train, X_test, y_train, y_test = train_test_split(paths, 
                                                  targets,
                                                  test_size=0.20, 
                                                  random_state=19)
train_generator = ImageGenerator(X_train, y_train, batch_size=32, shape=(224,224,3), augment=True)
test_generator = ImageGenerator(X_test, y_test, batch_size=32, shape=(224,224,3), augment=False)

In [ ]:
# Evaluate
score = model.evaluate_generator(generator=test_generator,
                                verbose=1,
                                steps=len(train_generator))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [ ]:
# Function to predict the dog breed and return the name
def predict_breed(filename):
    # open image, resize it
    img = Image.open(filename)
    img = img.convert('RGB')
    img = img.resize((224, 224))
    img.save(filename)
    # predict
    img = imread(filename)
    img = preprocess_input(img)
    probs = model.predict(np.expand_dims(img, axis=0))
    for idx in probs.argsort()[0][::-1][:1]:
        breed_pred = label_maps_rev[idx].split("-")[-1]
    return breed_pred

In [ ]:
predict_breed('static/test1.jpg')